In [1]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile
import os
from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()


# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpetg1huzk/zarr_dir'


In [2]:
# Run create-ome-zarr-multiplex and yokogawa-to-zarr
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_multiplex"],
            args=dict(image_dir="/tmp/input_images"),
        ),
        WorkflowTask(task=TASK_LIST["yokogawa_to_zarr"], args={}),
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Print current dataset information
debug(dataset);

[create_ome_zarr_multiplex] START
[create_ome_zarr_multiplex] image_dir='/tmp/input_images'
[create_ome_zarr_multiplex] zarr_dir='/tmp/tmpetg1huzk/zarr_dir'
[create_ome_zarr_multiplex] zarr_path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr'
[create_ome_zarr_multiplex] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_1.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/2'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_2.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] p

In [3]:
# Run init registration
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["init_registration"],
            args={"ref_acquisition": 0},
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset);

/tmp/ipykernel_120775/3673135805.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
            'init_registration',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/2',
                a

In [4]:
# Run calculate registration
dataset = execute_tasks_v2(
    wf_task_list=[WorkflowTask(task=TASK_LIST["calculate_registration"])],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset);

# In all non-reference-cycle images, a certain table was updated
for image in dataset.images:
    if image.attributes["acquisition"] == 0:
        assert not os.path.isfile(f"{image.path}/registration_table")
    else:
        assert os.path.isfile(f"{image.path}/registration_table")

/tmp/ipykernel_120775/458783815.py:9 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
            'init_registration',
            'calculate_registration',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_p

In [5]:
# Run find_registration_consensus
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(task=TASK_LIST["find_registration_consensus"])
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset);

# In all images, a certain (post-consensus) table was updated
for image in dataset.images:
    assert os.path.isfile(f"{image.path}/registration_table_final")

# The image list still has the original six images only
assert len(dataset.images) == 6

/tmp/ipykernel_120775/1697561531.py:11 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
            'init_registration',
            'calculate_registration',
            'find_registration_consensus',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
       

In [6]:
# Run apply_registration_to_image
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["apply_registration_to_image"],
            args={"overwrite_input": False},
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Images are now twice as many, and the second half of them are marked as registered
assert len(dataset.images) == 12
for image in dataset.images[:6]:
    assert "registration" not in image.attributes
for image in dataset.images[6:]:
    assert image.attributes["registration"] is True

# Print current dataset information
debug(dataset);

[registration] START
[registration] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1'
[registration] overwrite_input=False
[registration] Reading information from /tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1/registration_table_final
[registration] START
[registration] path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/02/0'
[registration] overwrite_input=False
[registration] Reading information from /tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/02/0/registration_table_final
[registration] new_path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1_r'
[registration] new_path='/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/02/0_r'
[registration] out={'added_images': [{'path': '/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/01/1_r'}]}
[registration] END
[registration] out={'added_images': [{'path': '/tmp/tmpetg1huzk/zarr_dir/my_plate.zarr/A/02/0_r'}]}
[registration] END
/tmp/ipykernel_120775/898703565.py:21 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_m